<a href="https://colab.research.google.com/github/charlotterempelmc/NLP-Week1-Text-Classification/blob/main/NLP_ShinCharlotte_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Hello, I'm **Wesley**, nice to meet you!👋

I was just reading the IMDb reviews of [*The Super Mario Bros. Movie*](https://www.imdb.com/title/tt6718170/), I thought why don't we make a **Sentiment Classifier** to categorize movie reviews! **WARNING: Spoilers ahead.**


Here we will be doing [transfer learning](https://en.wikipedia.org/wiki/Transfer_learning) on BERT [(blog)](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html) [(paper)](https://arxiv.org/abs/1810.04805v2) with an IMDb dataset to make a sentiment classifier for movie reviews.

# Setup Python Libraries (pip)

In [ ]:
pip install hf_transfer

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#install some Python packages with pip

!pip install numpy torch datasets transformers evaluate --quiet

In [ ]:
# let's check the version we are using

!pip freeze | grep -E '^numpy|^torch|^datasets|^transformers|^evaluate'

datasets==4.4.1
evaluate==0.4.6
numpy==2.1.2
torch==2.8.0+cu128
torchaudio==2.8.0+cu128
torchvision==0.23.0+cu128
transformers==4.57.1


# Create IMDB Dataset for Fine-tuning BERT

## Let's load the IMDB Dataset

In [ ]:
#load our text reviews dataset
from datasets import load_dataset

# let's load the imdb dataset from huggingface
# source: (https://huggingface.co/datasets/imdb) citizenlab/distilbert-base-multilingual-cased-toxicity

raw_dataset = load_dataset('Yelp/yelp_review_full')
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [ ]:
#load our text reviews dataset
from datasets import load_dataset

# let's load the imdb dataset from huggingface
# source: (https://huggingface.co/datasets/imdb) citizenlab/distilbert-base-multilingual-cased-toxicity

raw_datasetTest = load_dataset('charlotterempel/testsplit-finalproject')
raw_datasetTest

DatasetDict({
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 60
    })
})

## Let's create the train, validation, test sets

In [ ]:
# get train and validation set

dataset = raw_dataset['train'].train_test_split(test_size=0.2, seed=42, shuffle=True)
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 520000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 130000
    })
})

In [ ]:
# rename validation key to 'val'

dataset['val'] = dataset['test']
# dataset

In [ ]:
# copy test set from raw_dataset

dataset['test'] = raw_dataset['test']
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 520000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
    val: Dataset({
        features: ['label', 'text'],
        num_rows: 130000
    })
})

## We start by tokenizing our dataset with the BERT's Fast Tokenizer

In [ ]:
# import pretrained faster tokenizer from huggingface
# source: (https://huggingface.co/ElizaClaPa/SentimentAnalysis-YelpReviews-OptimizedModel)

from transformers import AutoTokenizer

checkpoint = 'ElizaClaPa/SentimentAnalysis-YelpReviews-OptimizedModel'
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True)
tokenizer

DistilBertTokenizerFast(name_or_path='ElizaClaPa/SentimentAnalysis-YelpReviews-OptimizedModel', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [ ]:
# tokenize the text in batches with truncation and padding based on BERT requirements

def tokenization(example):
    return tokenizer(example['text'], truncation=True, padding=True)

tokenized_dataset = dataset.map(tokenization, batched=True, remove_columns=['text'])
tokenized_dataset

Map:   0%|          | 0/130000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 520000
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
    val: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 130000
    })
})

In [ ]:
#tokenize test split
# tokenize the text in batches with truncation and padding based on BERT requirements

def tokenization(example):
    return tokenizer(example['text'], truncation=True, padding=True)

tokenized_datasetTest = raw_datasetTest.map(tokenization, batched=True, remove_columns=['text'])
tokenized_datasetTest

DatasetDict({
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 60
    })
})

# Setup Training Metrics (Accuracy, F1)

In [ ]:
!pip install scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
import evaluate
import numpy as np

# we setup the training to evaluate accuracy, precision, recall & f1 scores

accuracy_metric = evaluate.load('accuracy')
precision_metric = evaluate.load('precision')
recall_metric = evaluate.load('recall')
f1_metric = evaluate.load('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    precision = precision_metric.compute(predictions=predictions, references=labels, average="weighted")
    recall = recall_metric.compute(predictions=predictions, references=labels, average="weighted")
    return {**accuracy, **precision, **recall, **f1,}

# Setup Training Configurations

In [ ]:
pip install transformers[torch]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
#training 1 - multi-class sentiment analysis (fine-tuning Hugging Face model)
import os
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, IntervalStrategy

# get bert model with a sequence classification head for sentiment analysis
# source: (https://huggingface.co/ElizaClaPa/SentimentAnalysis-YelpReviews-OptimizedModel)
checkpoint = 'ElizaClaPa/SentimentAnalysis-YelpReviews-OptimizedModel'
num_labels = 5
id2label = {0:'1 star',1:'2 stars',2:'3 stars', 3:'4 stars',4:'5 stars'}
label2id = {'1 star':0,'2 stars':1,'3 stars':2,'4 stars':3,'5 stars':4}
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id)

# setup custom training arguments
# 1. store training checkpoints to 'results' output directory
# 2. fine-tune for just 1 epoch
# 3,4. use 16 as a batch size to speed things up
# 5. evaluate validation set every 500 steps (this is the default steps)
# 6. load the best model based on the lowest validation loss at the end of training
training_args = TrainingArguments(
    seed=42,
    output_dir = './results',
    num_train_epochs = 1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    load_best_model_at_end=True,
    eval_strategy = "epoch",
    save_strategy = 'epoch',
    learning_rate = 7e-6
)

# setup trainer with custom metrics (accuracy, precision, recall & f1)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['val'],
    compute_metrics=compute_metrics,
)

# disable wandb logging (a v4 huggingface artifact)
os.environ['WANDB_DISABLED']= "true"

In [ ]:
#training 2 - multi-class sentiment analysis (fine-tuning Hugging Face model)
import os
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, IntervalStrategy

# get bert model with a sequence classification head for sentiment analysis
# source: (https://huggingface.co/ElizaClaPa/SentimentAnalysis-YelpReviews-OptimizedModel)
checkpoint = 'ElizaClaPa/SentimentAnalysis-YelpReviews-OptimizedModel'
num_labels = 5
id2label = {0:'1 star',1:'2 stars',2:'3 stars', 3:'4 stars',4:'5 stars'}
label2id = {'1 star':0,'2 stars':1,'3 stars':2,'4 stars':3,'5 stars':4}
model2 = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id)

# setup custom training arguments
# 1. store training checkpoints to 'results' output directory
# 2. fine-tune for just 1 epoch
# 3,4. use 16 as a batch size to speed things up
# 5. evaluate validation set every 500 steps (this is the default steps)
# 6. load the best model based on the lowest validation loss at the end of training
training_args = TrainingArguments(
    seed=42,
    output_dir = './results',
    num_train_epochs = 1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    load_best_model_at_end=True,
    eval_strategy = "epoch",
    save_strategy = 'epoch',
    learning_rate = 3e-5
)

# setup trainer with custom metrics (accuracy, precision, recall & f1)
trainer2 = Trainer(
    model=model2,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['val'],
    compute_metrics=compute_metrics,
)

# disable wandb logging (a v4 huggingface artifact)
os.environ['WANDB_DISABLED']= "true"

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
#training 3 - multi-class sentiment analysis (fine-tuning Hugging Face model)
import os
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, IntervalStrategy

# get bert model with a sequence classification head for sentiment analysis
# source: (https://huggingface.co/ElizaClaPa/SentimentAnalysis-YelpReviews-OptimizedModel)
checkpoint = 'ElizaClaPa/SentimentAnalysis-YelpReviews-OptimizedModel'
num_labels = 5
id2label = {0:'1 star',1:'2 stars',2:'3 stars', 3:'4 stars',4:'5 stars'}
label2id = {'1 star':0,'2 stars':1,'3 stars':2,'4 stars':3,'5 stars':4}
model3 = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id)

# setup custom training arguments
# 1. store training checkpoints to 'results' output directory
# 2. fine-tune for just 1 epoch
# 3,4. use 16 as a batch size to speed things up
# 5. evaluate validation set every 500 steps (this is the default steps)
# 6. load the best model based on the lowest validation loss at the end of training
training_args = TrainingArguments(
    seed=42,
    output_dir = './results',
    num_train_epochs = 1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    load_best_model_at_end=True,
    eval_strategy = "epoch",
    save_strategy = 'epoch',
    learning_rate = 6e-8
)

# setup trainer with custom metrics (accuracy, precision, recall & f1)
trainer3 = Trainer(
    model=model3,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['val'],
    compute_metrics=compute_metrics,
)

# disable wandb logging (a v4 huggingface artifact)
os.environ['WANDB_DISABLED']= "true"

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Evaluate UnFine-Tuned BERT on Test Set for a Baseline Metric


In [ ]:
# evaluate unfine-tuned model with test set

trainer.evaluate(tokenized_dataset['test'])

{'eval_loss': 0.7893904447555542,
 'eval_model_preparation_time': 0.0023,
 'eval_accuracy': 0.68256,
 'eval_f1': 0.6835976156302593,
 'eval_runtime': 740.0072,
 'eval_samples_per_second': 67.567,
 'eval_steps_per_second': 4.223}

In [ ]:
# evaluate unfine-tuned model with own test split

trainer.evaluate(tokenized_datasetTest['train'])

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: charlotterempelmc (charlotterempel) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'eval_loss': 0.5010844469070435,
 'eval_model_preparation_time': 0.0014,
 'eval_accuracy': 0.8,
 'eval_precision': 0.7,
 'eval_recall': 0.8,
 'eval_f1': 0.7333333333333333,
 'eval_runtime': 0.8561,
 'eval_samples_per_second': 5.84,
 'eval_steps_per_second': 1.168}

Without fine-tuning BERT, our model currently has around **52% Accuracy (eval_accuracy)** and **19% F1 (eval_f1)**, which is pretty bad due to the test dataset having around 50% positive and 50% negative reviews. 😕


Let's make it better with transfer learning! 🦾

# Fine-Tune BERT with IMDb Dataset

In [ ]:
# fine-tune BERT with the yelp dataset (changed 1 hyperparameter: learning rate)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.468500,0.415966,0.841685,0.842009,0.841685,0.841766


TrainOutput(global_step=32500, training_loss=0.47291602313701925, metrics={'train_runtime': 6432.5914, 'train_samples_per_second': 80.838, 'train_steps_per_second': 5.052, 'total_flos': 6.88867325952e+16, 'train_loss': 0.47291602313701925, 'epoch': 1.0})

In [ ]:
trainer.save_model("trainedModelOne")

In [ ]:
trainer2.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.515900,0.475856,0.811900,0.812330,0.811900,0.812023


TrainOutput(global_step=32500, training_loss=0.5359185208834134, metrics={'train_runtime': 6410.7088, 'train_samples_per_second': 81.114, 'train_steps_per_second': 5.07, 'total_flos': 6.88867325952e+16, 'train_loss': 0.5359185208834134, 'epoch': 1.0})

In [ ]:
trainer2.save_model("trainedModelTwo")

In [ ]:
trainer2.evaluate(tokenized_datasetTest['test'])

{'eval_loss': 1.0076663494110107,
 'eval_accuracy': 0.6166666666666667,
 'eval_precision': 0.6640756302521008,
 'eval_recall': 0.6166666666666667,
 'eval_f1': 0.6039297416579363,
 'eval_runtime': 0.1563,
 'eval_samples_per_second': 383.825,
 'eval_steps_per_second': 25.588,
 'epoch': 1.0}

In [ ]:
##############################

In [ ]:
trainer3.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.484900,0.446563,0.826754,0.827290,0.826754,0.826839


TrainOutput(global_step=32500, training_loss=0.4873717017540565, metrics={'train_runtime': 6425.3172, 'train_samples_per_second': 80.93, 'train_steps_per_second': 5.058, 'total_flos': 6.88867325952e+16, 'train_loss': 0.4873717017540565, 'epoch': 1.0})

In [ ]:
trainer3.save_model("model3")

In [ ]:
# continue training trainer2
# trainer2.train(resume_from_checkpoint='/content/drive/MyDrive/checkpoints/checkpoint-1000')

	logging_steps: 500 (from args) != 5000 (from trainer_state.json)
	eval_steps: 500 (from args) != 5000 (from trainer_state.json)


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer2.evaluate(tokenized_dataset['test'])

In [ ]:
# let's see how well it did in the test set

trainer.evaluate(tokenized_dataset['test'])

In [ ]:
# trainer.save("myfinalmodel") #save & download it

**WOAH!** We got a **92% Accuracy (eval_accuracy)** and **92% F1 (eval_f1)** with just **1 epoch**! 🤯

# Try out some examples!

In [ ]:
from transformers import pipeline
import torch

# get current device with pytorch
device = torch.cuda.current_device()

# create pipeline for sentiment classifier with custom model and tokenizer
sentiment_classifier = pipeline(task='sentiment-analysis', model=model, tokenizer=tokenizer, device=device)

In [ ]:
# let's see how our model classifies a good review
# this is from 'justinvitelli' (https://www.imdb.com/review/rw8972952)

review = """
First off this movie is for kids and fans of Nintendo and the Mario franchise.
I still think an adult who isnt a fan could still enjoy it but this movie is so
full of fan service that it will have you smiling the whole time.
The voice acting I was skeptical but they all work and work well too.
Jack Black is the star here. I love how they kept the story simple like all of the games.
Truly felt like a video game on screen.
This movie felt like a beautifully animated amusement park ride.
The audio in the movie was amazing too.
The sounds and the score with reimagined iconic music was perfect.
Some of the songs in the movie felt unnecessary but they worked.
I think they should've bumped the run time to 105-120 min.
90 min felt too short as it goes by quick.
I havent had this much wholesome fun at the movies in a long time.
If youre a fan you HAVE to see it.
"""
sentiment_classifier(review)

[{'label': 'POSITIVE', 'score': 0.9938808679580688}]

That is **99% POSITIVE**! *justinvitelli* loves the movie!

In [ ]:
# let's see how our model classifies a bad review
# this is from 'industriousbug16' (https://www.imdb.com/review/rw8998214)

review = """
Flat, visual noise.
Fundamentally incurious. Potentially injurious.
The mystique generated by the characters in the games is here raked over and presented
haphazardly by hacks.
A hobbled attempt to explain a long and random evolution of characters who were never meant
to be narratised fails.
Doing it well is near impossible when you insist on EVERY LITTLE BIT OF LORE,
from the last forty years being shoehorned into 90 minutes.
Makes little sense, shamelessly leans on member berries to stimulate older viewers but offers
nothing else.
I feel sad for the animators who did a sterling job, but to no end as this movie has no soul.
"""
sentiment_classifier(review)

[{'label': 'NEGATIVE', 'score': 0.9951890707015991}]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

checkpoint = "charlotterempel/fine-tuned-shinCharlotte"

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sentiment_classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=device)

sentiment_classifier("HORRIBLE SERVICE. Thank you for giving me absolute hell.")

Device set to use cuda:0


[{'label': '1 star', 'score': 0.9955578446388245}]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

checkpoint = "charlotterempel/fine-tuned-shinCharlotte"

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sentiment_classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=device)

sentiment_classifier("DISGUSTING. What an absolute nightmare...")

Device set to use cuda:0


[{'label': '1 star', 'score': 0.9944145679473877}]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

checkpoint = "charlotterempel/fine-tuned-shinCharlotte"

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sentiment_classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=device)

sentiment_classifier("These sunglasses are the most beautiful thing i have ever worn, you won't regret it!")

Device set to use cuda:0


[{'label': '5 stars', 'score': 0.97735196352005}]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

checkpoint = "charlotterempel/fine-tuned-shinCharlotte"

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sentiment_classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=device)

sentiment_classifier("This leather jacket was pretty good. But for the price, i wish it didn't smell bad but overall decent.")

Device set to use cuda:0


[{'label': '3 stars', 'score': 0.8761754631996155}]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

checkpoint = "charlotterempel/fine-tuned-shinCharlotte"

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sentiment_classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=device)

sentiment_classifier("mid bro.")

Device set to use cuda:0


[{'label': '2 stars', 'score': 0.49382418394088745}]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

checkpoint = "charlotterempel/fine-tuned-shinCharlotte"

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sentiment_classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=device)

sentiment_classifier("This was bangin, absolute smash! Wowzas.")

In [6]:
!jupyter nbconvert --ClearMetadataPreprocessor.enabled=True \
                   --to notebook \
                   --output=clean.ipynb \
                   NLP_ShinCharlotte_Project.ipynb


[NbConvertApp] WARNING | pattern 'NLP_ShinCharlotte_Project.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]

That is **99% NEGATIVE**! *industriousbug16* must hate the movie very badly.

# Resources

### If you would like to use this model without running the entire notebook, try the model at my [HuggingFace](https://huggingface.co/wesleyacheng/movie-review-sentiment-classifier-with-bert).

### If you woud like to get this in GitHub, here's my [repo](https://github.com/wesleyacheng/movie-review-sentiment-classifier-with-bert).